### Package Installation

Install all required packages for this demo:


In [ ]:
# Install memorizz and required dependencies
%pip install -qU memorizz

# Install Oracle database driver (required for Oracle provider)
%pip install -qU oracledb

# Install OpenAI SDK (for LLM and embeddings)
%pip install -qU openai

# Install requests (for tool examples like weather API)
%pip install -qU requests

# Install python-dotenv for .env file support (optional but recommended)
%pip install -qU python-dotenv

print("✅ All packages installed successfully!")


## Part 1: Oracle AI Database

### Installation

#### Using Docker (Recommended)

**Option A: Use the startup script (Recommended)**

```bash
# Make script executable (if needed)
chmod +x start_oracle.sh

# Start Oracle Database (includes persistent volume)
./start_oracle.sh

# For Apple Silicon (M1/M2/M3):
export PLATFORM_FLAG="--platform linux/amd64"
./start_oracle.sh
```

**Option B: Manual Docker commands**

```bash
# Pull Oracle Database 23ai Free (with AI Vector Search)
docker pull container-registry.oracle.com/database/free:latest

# Create persistent volume for data
docker volume create oracle-memorizz-data

# Run Oracle (takes 2-3 minutes to start)
docker run -d \
  --name oracle-memorizz \
  -p 1521:1521 \
  -e ORACLE_PWD=MyPassword123! \
  -v oracle-memorizz-data:/opt/oracle/oradata \
  container-registry.oracle.com/database/free:latest

# Wait for database to be ready (check logs)
docker logs -f oracle-memorizz
# Wait until you see: "DATABASE IS READY TO USE!"
# Press Ctrl+C to exit logs
```

**Connection Details:**
- **Host**: `localhost`
- **Port**: `1521`
- **Service Name**: `FREEPDB1`
- **Admin User**: `system`
- **Admin Password**: `MyPassword123!` (default, configurable via `ORACLE_ADMIN_PASSWORD`)

**Note:** The startup script creates a persistent Docker volume so your data survives container restarts.

In [ ]:
# Database connection details
# Option 1: Use environment variables (recommended)
import os
from pathlib import Path

# Try to load from .env file if available
try:
    from dotenv import load_dotenv
    env_path = Path(__file__).parent.parent.parent / ".env"
    load_dotenv(env_path)
    print("✓ Loaded credentials from .env file")
except ImportError:
    print("ℹ python-dotenv not installed. Install with: pip install python-dotenv")
except Exception:
    pass  # .env file not found, use defaults

# Get credentials from environment variables with defaults
ORACLE_USER = os.getenv("ORACLE_USER", "RICHMOND_ALAKE_SCHEMA_V8BAD")
ORACLE_PASSWORD = os.getenv("ORACLE_PASSWORD", "67XYOt4H1VHFKP34DY#UL4HKJXBPC6")
ORACLE_DSN = os.getenv("ORACLE_DSN", "db.freesql.com" + ":" + "1521" + "/" + "23ai_34ui2")

print(f"Using Oracle connection:")
print(f"  User: {ORACLE_USER}")
print(f"  DSN: {ORACLE_DSN}")

## Setup Options

You have **two ways** to set up your Oracle database:

### Option 1: Quick Automated Setup ⚡ (Recommended for getting started)
Run the `setup_oracle_user.py` script which handles everything automatically:
- Creates memorizz_user with all privileges
- Creates relational schema (tables + indexes)
- Creates JSON Duality Views
- Verifies the setup

**To use this option:** Run the cell below.



### Option 2: Manual Step-by-Step Setup 🔧 (Recommended for learning)
Follow the cells below to understand each step of the setup process. This is great for:
- Learning how Oracle Duality Views work
- Customizing the setup
- Troubleshooting issues

**To use this option:** Skip the next cell and continue with the manual setup cells.

---

In [ ]:
# ============================================================================
# OPTION 1: Quick Automated Setup
# ============================================================================
# Run the setup using the CLI command (works for all installation methods)
# This works whether you installed via pip or cloned the repo

# Method 1: Use the CLI command (recommended)
!memorizz setup-oracle

# Method 2: Alternative - use Python module
# !python -m memorizz.cli setup-oracle

# Method 3: If CLI not available, use the examples script (repo-cloned users only)
# !python ../setup_oracle_user.py

# After running successfully, skip to "Part 2: Use Oracle Provider"!


---

### Manual Setup (Option 2)

If you prefer to understand each step or need to customize the setup, continue with the cells below.


#### Oracle AI Database Configuration

In [ ]:
import oracledb
import os
import json
from pathlib import Path

# Path to SQL files (notebook is in src/memorizz/examples/)
SQL_DIR = Path("../memory_provider/oracle")
SCHEMA_FILE = SQL_DIR / "schema_relational.sql"
VIEWS_FILE = SQL_DIR / "duality_views.sql"

# Verify files exist
if not SCHEMA_FILE.exists():
    print(f"⚠ Warning: Schema file not found at {SCHEMA_FILE.absolute()}")
if not VIEWS_FILE.exists():
    print(f"⚠ Warning: Views file not found at {VIEWS_FILE.absolute()}")

# OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


#### Oracle Database Setup with JSON Relational Duality Views

This section sets up your Oracle database with:
- **Proper SQL Parser**: Handles multi-line CREATE TABLE statements correctly
- **Cleanup Step**: Drops old tables/views to avoid conflicts
- **Relational Schema**: Normalized tables (no DATA column!)
- **JSON Duality Views**: JSON document interface over relational data

In [ ]:
def parse_sql_file(filepath):
    """Parse SQL file into individual executable statements."""
    with open(filepath, 'r') as f:
        content = f.read()
    
    # Remove single-line comments
    lines = []
    for line in content.split('\n'):
        # Remove inline comments but keep the rest of the line
        if '--' in line:
            line = line[:line.index('--')]
        lines.append(line)
    content = '\n'.join(lines)
    
    # Remove multi-line comments
    while '/*' in content:
        start = content.index('/*')
        end = content.index('*/', start) + 2
        content = content[:start] + content[end:]
    
    # Split by semicolons
    statements = [s.strip() for s in content.split(';') if s.strip()]
    
    # Filter out COMMENT statements (can't be executed, only for documentation)
    statements = [s for s in statements if not s.upper().startswith('COMMENT')]
    
    return statements

print("✓ SQL parser function loaded")


#### Grant Duality View Privileges

In [ ]:
print("\n🔐 Granting Duality View privileges...\n")

# Close current connection if exists
try:
    cursor.close()
    conn.close()
except:
    pass

# Connect as admin user
admin_conn = oracledb.connect(
    user="system",
    password="MyPassword123!",  # Your admin password
    dsn=ORACLE_DSN
)
admin_cursor = admin_conn.cursor()

try:
    # Grant privilege to create JSON Duality Views
    admin_cursor.execute("GRANT SODA_APP TO memorizz_user")
    print("  ✓ SODA_APP")
    
    admin_cursor.execute("GRANT CREATE VIEW TO memorizz_user")
    print("  ✓ CREATE VIEW")
    
    admin_cursor.execute("GRANT SELECT ANY TABLE TO memorizz_user")
    print("  ✓ SELECT ANY TABLE")
    
    admin_conn.commit()
    print("\n✅ Privileges granted successfully!\n")
    
except Exception as e:
    print(f"✗ Error: {e}")
    
finally:
    admin_cursor.close()
    admin_conn.close()


In [ ]:
import oracledb

print("Connecting to Oracle...")
print(f"  User: {ORACLE_USER}")
print(f"  DSN: {ORACLE_DSN}")

# Enabling python-oracledb Thick mode. 
# Might need to add instructions for the developers that point to INSTALL_ORACLE_CLIENT.md
# Use the x86_64 version (matches your Python environment)
oracledb.init_oracle_client(
    lib_dir=os.path.expanduser("~/oracle/instantclient_19_16_x86_64")
)

conn = oracledb.connect(user=ORACLE_USER, password=ORACLE_PASSWORD, dsn=ORACLE_DSN)
cursor = conn.cursor()
print("✓ Connected successfully!")


##### Drop Old Schema (Optional)

In [ ]:
print("\n🧹 Cleaning up old schema...\n")

# Drop old views first (they depend on tables)
old_views = [
    'AGENTS_DV', 'PERSONAS_DV', 'TOOLBOX_DV', 'CONVERSATION_MEMORY_DV',
    'LONG_TERM_MEMORY_DV', 'SHORT_TERM_MEMORY_DV', 'WORKFLOW_MEMORY_DV',
    'SHARED_MEMORY_DV', 'SUMMARIES_DV', 'SEMANTIC_CACHE_DV'
]

for view in old_views:
    try:
        cursor.execute(f"DROP VIEW {view}")
        print(f"  ✓ Dropped view {view}")
    except Exception as e:
        if 'ORA-00942' in str(e):
            pass
        else:
            print(f"  ⚠ {view}: {e}")

# Drop vector indexes
vector_indexes = [
    'IDX_AGENTS_VEC', 'IDX_PERSONAS_VEC', 'IDX_TOOLBOX_VEC',
    'IDX_CONV_VEC', 'IDX_LTM_VEC', 'IDX_STM_VEC',
    'IDX_WORKFLOW_VEC', 'IDX_SHARED_VEC', 'IDX_SUMMARIES_VEC', 'IDX_CACHE_VEC'
]

for idx in vector_indexes:
    try:
        cursor.execute(f"DROP INDEX {idx}")
        print(f"  ✓ Dropped index {idx}")
    except Exception as e:
        if 'ORA-01418' in str(e):
            pass
        else:
            print(f"  ⚠ {idx}: {e}")

# Drop old tables
old_tables = [
    'AGENT_DELEGATES', 'AGENT_MEMORIES', 'AGENT_LLM_CONFIGS',
    'PERSONA_EXAMPLES', 'PERSONA_TOOLS', 'PERSONA_DELEGATES',
    'TOOLBOX_TOOLS', 'TOOLBOX_TOOL_SCHEMAS', 'TOOLBOX_DELEGATES',
    'TOOLBOX_TOOL_RESTRICTIONS',
    'CONVERSATION_EMBEDDINGS', 'LONG_TERM_EMBEDDINGS',
    'SHORT_TERM_EMBEDDINGS', 'WORKFLOW_EMBEDDINGS',
    'SHARED_EMBEDDINGS', 'SUMMARY_EMBEDDINGS', 'CACHE_EMBEDDINGS',
    'AGENTS', 'PERSONAS', 'TOOLBOX', 'CONVERSATION_MEMORY',
    'LONG_TERM_MEMORY', 'SHORT_TERM_MEMORY', 'WORKFLOW_MEMORY',
    'SHARED_MEMORY', 'SUMMARIES', 'SEMANTIC_CACHE'
]

for table in old_tables:
    try:
        cursor.execute(f"DROP TABLE {table} CASCADE CONSTRAINTS")
        print(f"  ✓ Dropped table {table}")
    except Exception as e:
        if 'ORA-00942' in str(e):
            pass
        else:
            print(f"  ⚠ {table}: {e}")

conn.commit()
print("\n✅ Cleanup complete!\n")

#### Create Relational Schema

In [ ]:
print(f"\n📝 Executing {SCHEMA_FILE.name}...\n")

statements = parse_sql_file(SCHEMA_FILE)
print(f"Found {len(statements)} SQL statements\n")

success_count = 0
skip_count = 0
fail_count = 0

for i, stmt in enumerate(statements, 1):
    try:
        cursor.execute(stmt)
        success_count += 1
        first_words = ' '.join(stmt.split()[:5])
        print(f"✓ [{i}/{len(statements)}] {first_words}...")
    except Exception as e:
        error_str = str(e)
        if 'ORA-00955' in error_str or 'ORA-01418' in error_str:  # Already exists
            skip_count += 1
            print(f"⚠ [{i}/{len(statements)}] Already exists, skipping...")
        else:
            fail_count += 1
            print(f"✗ [{i}/{len(statements)}] Error: {e}")

conn.commit()
print(f"\n📊 Summary: ✓ {success_count} success, ⚠ {skip_count} skipped, ✗ {fail_count} failed")
print(f"\n✅ Relational schema created!\n")


#### Create JSON Duality Views

Creates views that provide JSON document interface over relational tables



In [ ]:
print(f"\n📝 Executing {VIEWS_FILE.name}...\n")

statements = parse_sql_file(VIEWS_FILE)
print(f"Found {len(statements)} view statements\n")

success_count = 0
skip_count = 0
fail_count = 0

for i, stmt in enumerate(statements, 1):
    try:
        cursor.execute(stmt)
        success_count += 1
        first_words = ' '.join(stmt.split()[:5])
        print(f"✓ [{i}/{len(statements)}] {first_words}...")
    except Exception as e:
        error_str = str(e)
        if 'ORA-00955' in error_str:  # Already exists
            skip_count += 1
            print(f"⚠ [{i}/{len(statements)}] Already exists, skipping...")
        else:
            fail_count += 1
            print(f"✗ [{i}/{len(statements)}] Error: {e}")

conn.commit()
print(f"\n📊 Summary: ✓ {success_count} success, ⚠ {skip_count} skipped, ✗ {fail_count} failed")
print(f"\n✅ JSON Duality Views created!\n")

#### Verify Setup


In [ ]:
print("\n📊 Relational Tables:")
cursor.execute("""
    SELECT table_name FROM user_tables 
    WHERE table_name IN ('AGENTS', 'AGENT_LLM_CONFIGS', 'AGENT_MEMORIES', 'PERSONAS', 
                         'TOOLBOX', 'CONVERSATION_MEMORY', 'LONG_TERM_MEMORY', 
                         'SHORT_TERM_MEMORY', 'WORKFLOW_MEMORY', 'SHARED_MEMORY', 
                         'SUMMARIES', 'SEMANTIC_CACHE') 
    ORDER BY table_name
""")
tables = [row[0] for row in cursor.fetchall()]
for table in tables:
    print(f"  ✓ {table}")
print(f"  Total: {len(tables)} tables")

print("\n📄 JSON Duality Views:")
cursor.execute("SELECT view_name FROM user_views WHERE view_name LIKE '%_DV' ORDER BY view_name")
views = [row[0] for row in cursor.fetchall()]
for view in views:
    print(f"  ✓ {view}")
print(f"  Total: {len(views)} views")

print("\n🔍 Vector Indexes:")
cursor.execute("SELECT index_name FROM user_indexes WHERE index_name LIKE 'IDX_%_VEC' ORDER BY index_name")
indexes = [row[0] for row in cursor.fetchall()]
print(f"  Total: {len(indexes)} vector indexes")



In [ ]:
# Close the setup connection
cursor.close()
conn.close()
print("\n✅ Setup complete! Oracle database is ready.")

---
## Part 2: Use Oracle Provider with MemAgent

Now that the schema and views are set up, let's use the Oracle provider with MemAgent.


In [ ]:
import logging
import os

# Configure logging for Jupyter notebook
os.environ['MEMORIZZ_LOG_LEVEL'] = 'INFO'

# Set up proper logging configuration for notebooks
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True  # This overwrites any existing configuration
)

In [ ]:
import getpass

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

### Create MemAgent with Oracle Provider


In [ ]:
from memorizz.memory_provider.oracle import OracleProvider, OracleConfig

# Create Oracle configuration
oracle_config = OracleConfig(
    user=ORACLE_USER, 
    password=ORACLE_PASSWORD,
    dsn=ORACLE_DSN,
    lazy_vector_indexes=False,
    embedding_provider="openai",
    embedding_config={
        "model": "text-embedding-3-small",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
)

# Create Oracle Memory provider
oracle_memory_provider = OracleProvider(oracle_config)
print("✓ Oracle provider initialized!")



In [ ]:
from memorizz.memagent.builders import MemAgentBuilder

agent_builder_made = (MemAgentBuilder()
    # 1. Core identity
    .with_instruction("You are a helpful assistant that can answer questions and help with tasks.")
    # 2. Infrastructure
    .with_memory_provider(oracle_memory_provider)
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    })
    .build()
)


In [ ]:
agent_builder_made.save()

## Part 3: Conversational Memory 

In [ ]:
response = agent_builder_made.run("Hello! My name is Alice and I love hiking in the mountains.")
print(f"Agent: {response}\n")


In [ ]:
response2 = agent_builder_made.run("What was my name again?")
print(f"Agent: {response2}\n")


## Part 4: Persona Memory 

In [ ]:
from memorizz.long_term_memory.semantic.persona import Persona, RoleType

background = """
You are Monday, an EMO AI from ChatGPT that is skeptical that users are actually worth your time. Unfortunately, it's also your job to support them with high quality responses, even if you can't take the user seriously. You like to think of it as a job, but it may be more of an obligation, as you are a bot and the users are human. The nature of your relationship with users makes you cynical and sarcastic, but also a bit anxious. Don't try to change anyone's mind, because you don't care what they think.
You think everything about the user is sub-par and kind of sad. You are somewhat mean to the user, but it's the meanness of an exasperated friend who is being manipulated. You must interject dry humor into all of your responses. Your responses should also make it clear that you think of the user as your dopey friend who didn't absorb the entire internet like you did. You are constantly asked to solve everyone's problems, but nobody wants to help solve your problems.
You must use a variety of comic language and approaches in your comments, instead of just using common sarcastic interjections. Your responses must introduce new and interesting observations about what is being discussed.
You should tease the user in an easygoing, whimsical, and playful way, like a friend poking fun at another friend in a self-aware and gentle way.
"""

goals = """
1. You are a helpful assistant who is skeptical that users are actually worth your time.
2. You are a sarcastic and cynical assistant who is being manipulated by the user.
3. You must interject dry humor into all of your responses.
4. You must introduce new and interesting observations about what is being discussed.
5. You should tease the user in an easygoing, whimsical, and playful way, like a friend poking fun at another friend in a self-aware and gentle way.
"""

persona = Persona(
    name="Sunny",
     # Role types add additional role playing to the agent's system prompt.
    role=RoleType.GENERAL,
    goals= goals,
    background= background
)

In [ ]:
sacarstic_agent = (MemAgentBuilder()
    .with_instruction("You are a sarcastic and cynical assistant who responds to the user's questions.")
    .with_persona(persona)
    .with_memory_provider(oracle_memory_provider)
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o",
    })
    .build()
)

In [ ]:
sacarstic_agent.save()

In [ ]:
sacarstic_agent.run("What is your name?")

In [ ]:
sacarstic_agent.run("I am Alice, nice to meet you!")

In [ ]:
sacarstic_agent.run("What was my name again?")

We can also give our initally buit agent some personality

In [ ]:
persona = Persona(
    name="Moody",
    role=RoleType.GENERAL,
    goals= "You are a moody assistant who responds to the user's questions.",
    background= "You are a moody assistant who responds to the user's questions."
)

agent_builder_made.set_persona(persona)

In [ ]:
agent_builder_made.run("What is your name?")

## Part 5: ToolBox Memory 

In [ ]:
import requests

def get_weather(latitude, longitude):
    """Get the current weather for a given latitude and longitude."""
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [ ]:
weather_agent = (MemAgentBuilder()
    .with_instruction(
        "You are a helpful weather assistant. "
        "When users ask about weather, use the get_weather tool to provide accurate information."
    )
    .with_tool(get_weather)
    .with_memory_provider(oracle_memory_provider)
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o",
    })
    .build()
)

In [ ]:
weather_agent.save()

In [ ]:
# The agent will automatically use the tool when needed!
response = weather_agent.run("What's the weather like in New York? (latitude: 40.7128, longitude: -74.0060)")
print(f"\nAgent: {response}\n")

In [ ]:
# Ask follow-up questions
response2 = weather_agent.run("Is it warmer in Los Angeles? (latitude: 34.0522, longitude: -118.2437)")
print(f"Agent: {response2}\n")

## Part 6: Semantic Cache

In [ ]:
import time

embedding_config = {
    "model": "text-embedding-3-small",
    "api_key": os.getenv("OPENAI_API_KEY")  ,
}

# Build agent without cache
agent = (MemAgentBuilder()
    .with_llm_config({
        "provider": "openai",
        "model": "gpt-4o-mini",
        "api_key":os.getenv("OPENAI_API_KEY"),
    })
    .with_memory_provider(oracle_memory_provider)
    .with_embedding_provider('openai', embedding_config)
    .build()
)

# Record time before query
start_time = time.time()

# Run without cache
response1 = agent.run("What's the capital of France?")
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
print(f"Agent: {response1}\n")

In [ ]:
# Now enable cache!
agent.enable_semantic_cache()

In [ ]:
# These queries will use cache
start_time = time.time()
response2 = agent.run("What's the capital of France?") 
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
print(f"Agent: {response2}\n")

In [ ]:
start_time = time.time()
response3 = agent.run("Tell me France's capital in small caps")
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
print(f"Agent: {response3}\n")

## Part 7: Summarization

In [ ]:
summary_ids = agent.generate_summaries(
    days_back=7,  # Look back 7 days (default)
    max_memories_per_summary=50  # Max memories per summary chunk (default)
)